In [1]:
!deactivate # if you're using any 
import os
import multiprocessing
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # Change if GPU is more than 1

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.metrics import error_rate
from fastai import *
from fastai.callbacks import *

import cv2 as cv
import numpy as np
import pandas as pd
import scipy.io as sio

torch.backends.cudnn.benchmark = True

In [3]:
tfms = get_transforms(
    do_flip=False, 
    flip_vert=True, 
    max_rotate=15.0
)

num_workers = multiprocessing.cpu_count()

In [4]:
def data_build(size):
    data = ImageDataBunch.from_folder(
        'data-kaggle/car_data/car_data/train',
        train='train',
        valid_pct=.2,
        ds_tfms=tfms,
        size=(size, size),
        num_workers=num_workers,
        bs=32).normalize(imagenet_stats)
    
    return data

In [5]:
def change_size(model, size=280):
    data = data_build(size)
    learn.data = data

In [6]:
model = models.resnet50

In [7]:
data_400 = data_build(400)

In [8]:
learn = cnn_learner(
    data_400, 
    model, 
    ps=0.1,
    bn_final=True,
    pretrained=True,
    opt_func=optim.SGD,
    metrics=[accuracy, error_rate]).mixup()

In [9]:
model_path = Path('./models')

In [10]:
learn.path = model_path

In [11]:
learn.load('93.mdl')

Learner(data=ImageDataBunch;

Train: LabelList (27614 items)
x: ImageList
Image (3, 400, 400),Image (3, 400, 400),Image (3, 400, 400),Image (3, 400, 400),Image (3, 400, 400)
y: CategoryList
Audi TT RS Coupe 2012,Audi TT RS Coupe 2012,Audi TT RS Coupe 2012,Audi TT RS Coupe 2012,Audi TT RS Coupe 2012
Path: data-kaggle/car_data/car_data/train;

Valid: LabelList (6903 items)
x: ImageList
Image (3, 400, 400),Image (3, 400, 400),Image (3, 400, 400),Image (3, 400, 400),Image (3, 400, 400)
y: CategoryList
Infiniti QX56 SUV 2011,Dodge Charger Sedan 2012,Nissan NV Passenger Van 2012,GMC Savana Van 2012,Bentley Mulsanne Sedan 2011
Path: data-kaggle/car_data/car_data/train;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=Fals

In [12]:
test_tfms = get_transforms()

In [14]:
data_test = ImageDataBunch.from_folder(
    'data-kaggle/car_data/car_data/test',
    valid_pct=0,
    ds_tfms=test_tfms,
    size=(500, 500),
    num_workers=num_workers,
    bs=32).normalize(imagenet_stats)

In [15]:
valid_dl = learn.data.valid_dl

In [16]:
test_dl = data_test.train_dl

In [17]:
learn.data.valid_dl = test_dl

In [18]:
learn.validate()

[0.3566219, tensor(0.9302), tensor(0.0698)]